In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentimental-analysis-for-tweets/sentiment_tweets3.csv


In [2]:
data = pd.read_csv("/kaggle/input/sentimental-analysis-for-tweets/sentiment_tweets3.csv")
data.head()

,Index,message to examine,label (depression result)
0,106,just had a real good moment. i missssssssss hi...,0
1,217,is reading manga http://plurk.com/p/mzp1e,0
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,288,@lapcat Need to send 'em to my accountant tomo...,0
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0


In [3]:
data.describe()

,Index,label (depression result)
count,10314.000000,10314.000000
mean,491253.470525,0.224355
std,261688.134407,0.417177
min,106.000000,0.000000
25%,263019.250000,0.000000
50%,514988.000000,0.000000
75%,773521.500000,0.000000
max,802313.000000,1.000000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10314 entries, 0 to 10313
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Index                      10314 non-null  int64 
 1   message to examine         10314 non-null  object
 2   label (depression result)  10314 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 241.9+ KB


In [5]:
data.isnull().sum()

Index                        0
message to examine           0
label (depression result)    0
dtype: int64

In [6]:
data = pd.DataFrame.rename(data, columns={'message to examine':'text'})

In [7]:
X = data['text']
y = data['label (depression result)']

In [8]:
X

0        just had a real good moment. i missssssssss hi...
1               is reading manga  http://plurk.com/p/mzp1e
2        @comeagainjen http://twitpic.com/2y2lx - http:...
3        @lapcat Need to send 'em to my accountant tomo...
4            ADD ME ON MYSPACE!!!  myspace.com/LookThunder
                               ...                        
10309    No Depression by G Herbo is my mood from now o...
10310    What do you do when depression succumbs the br...
10311    Ketamine Nasal Spray Shows Promise Against Dep...
10312    dont mistake a bad day with depression! everyo...
10313                                                    0
Name: text, Length: 10314, dtype: object

In [9]:
y

0        0
1        0
2        0
3        0
4        0
        ..
10309    1
10310    1
10311    1
10312    1
10313    1
Name: label (depression result), Length: 10314, dtype: int64

In [10]:
message = data.copy()

In [11]:
message

,Index,text,label (depression result)
0,106,just had a real good moment. i missssssssss hi...,0
1,217,is reading manga http://plurk.com/p/mzp1e,0
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,288,@lapcat Need to send 'em to my accountant tomo...,0
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0
...,...,...,...
10309,802309,No Depression by G Herbo is my mood from now o...,1
10310,802310,What do you do when depression succumbs the br...,1
10311,802311,Ketamine Nasal Spray Shows Promise Against Dep...,1
10312,802312,dont mistake a bad day with depression! everyo...,1


In [12]:
import nltk
from nltk.corpus import stopwords
import re

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
 message['text'][1]

'is reading manga  http://plurk.com/p/mzp1e'

In [15]:
# data Preprocessing
from nltk.stem import PorterStemmer
ps = PorterStemmer()
corpus = []

for i in range(0, len(message)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', message['text'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word)for word in review if not word in stopwords.words('english')]
    review =  ' '.join(review)
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [16]:
corpus

['real good moment missssssssss much',
 'read manga http plurk com p mzp e',
 'comeagainjen http twitpic com lx http www youtub com watch v zogfqvh',
 'lapcat need send em account tomorrow oddli even refer tax support evid though',
 'add myspac myspac com lookthund',
 'sleepi good time tonight though',
 'silkcharm nbn someon alreadi said fiber home mean least regular',
 'c possibl today nice',
 'nite twittervil workout ciao',
 'danann night darlin sweet dream',
 'good morn everybodi',
 'final creat wordpress blog alreadi blog seattl coffe commun http tinyurl com c uufd',
 'kisha cnt get u til get frm u rememb ur top',
 'nicolerichi ye rememb band awesom pleas repli',
 'realli love reflect shadow',
 'blueaero ooo fantasi like fantasi novel check',
 'rokchic prob sell noth blog http snedwan com get listen band itun',
 'shipovalov quot nokla connect peopl quot',
 'stay late start earli good thing like job',
 'kal penn read new job congratul fantast',
 'abl sleep think watch ugli betti onl

In [17]:
import numpy as np
import tensorflow
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout

In [18]:
voc_size = 10000

In [19]:
onehot_repr = [one_hot(words, voc_size)for words in corpus]
onehot_repr

[[7471, 6027, 4995, 2889, 571],
 [9715, 5015, 1560, 5626, 4640, 3491, 2928, 920],
 [5446, 1560, 3069, 4640, 2645, 1560, 5932, 3284, 4640, 619, 9981, 7172],
 [1545, 6880, 7181, 3664, 5931, 1199, 6025, 2526, 1930, 1044, 13, 2832, 4372],
 [4416, 7779, 7779, 4640, 9375],
 [573, 6027, 5704, 789, 4372],
 [1781, 7998, 3305, 7302, 4195, 7388, 9230, 6898, 3362, 4872],
 [116, 8968, 4523, 4057],
 [9362, 6339, 1769, 6934],
 [6896, 4105, 2940, 5993, 9391],
 [6027, 5106, 6706],
 [9549,
  4417,
  2410,
  9901,
  7302,
  9901,
  306,
  8786,
  4600,
  1560,
  7116,
  4640,
  116,
  5163],
 [9618, 6679, 5338, 6369, 3441, 5338, 5098, 6369, 710, 7012, 8186],
 [7756, 4266, 710, 4042, 4076, 4868, 3079],
 [8201, 3220, 3630, 2515],
 [3771, 6437, 7578, 5645, 7578, 3612, 9443],
 [9387, 5784, 6267, 1260, 9901, 1560, 1536, 4640, 5338, 276, 4042, 1758],
 [7447, 3330, 8940, 2058, 866, 3330],
 [2324, 7748, 6402, 6564, 6027, 614, 5645, 8422],
 [690, 4771, 9715, 6781, 8422, 3612, 6098],
 [5275, 662, 2071, 619, 4511, 

In [20]:
sent_len=20
word_embedding = pad_sequences(onehot_repr, sent_len)

In [21]:
word_embedding

array([[   0,    0,    0, ..., 4995, 2889,  571],
       [   0,    0,    0, ..., 3491, 2928,  920],
       [   0,    0,    0, ...,  619, 9981, 7172],
       ...,
       [   0,    0,    0, ..., 1560, 3405, 7336],
       [   0,    0,    0, ..., 7044, 2456, 3664],
       [   0,    0,    0, ...,    0,    0,    0]], dtype=int32)

In [22]:
feature_len = 40

In [23]:
feature_size = 40
model = Sequential()

model.add(Embedding(voc_size, feature_size, input_length=sent_len))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            400000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________


In [24]:
X_new = np.array(word_embedding)
y_new = np.array(y)

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.3)

In [26]:
X_train.shape

(7219, 20)

In [27]:
X_test.shape

(3095, 20)

In [28]:
X_train

array([[   0,    0,    0, ..., 1517, 9097, 2830],
       [   0,    0,    0, ..., 9384, 3283, 4868],
       [   0,    0,    0, ..., 4640, 6724, 7724],
       ...,
       [   0,    0,    0, ..., 8627, 3656, 4523],
       [4600, 3458, 7460, ..., 4640, 5921, 4551],
       [   0,    0,    0, ..., 1268, 2712, 4359]], dtype=int32)

In [29]:
y_train

array([0, 0, 0, ..., 0, 1, 0])

In [30]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
113/113 [==============================] - 6s 38ms/step - loss: 0.4782 - accuracy: 0.7970 - val_loss: 0.2149 - val_accuracy: 0.9325
Epoch 2/10
113/113 [==============================] - 4s 32ms/step - loss: 0.0592 - accuracy: 0.9801 - val_loss: 0.0377 - val_accuracy: 0.9919
Epoch 3/10
113/113 [==============================] - 3s 30ms/step - loss: 0.0099 - accuracy: 0.9979 - val_loss: 0.0613 - val_accuracy: 0.9822
Epoch 4/10
113/113 [==============================] - 3s 30ms/step - loss: 0.0092 - accuracy: 0.9987 - val_loss: 0.0428 - val_accuracy: 0.9893
Epoch 5/10
113/113 [==============================] - 3s 31ms/step - loss: 0.0064 - accuracy: 0.9985 - val_loss: 0.0337 - val_accuracy: 0.9919
Epoch 6/10
113/113 [==============================] - 3s 31ms/step - loss: 0.0040 - accuracy: 0.9994 - val_loss: 0.0409 - val_accuracy: 0.9919
Epoch 7/10
113/113 [==============================] - 3s 31ms/step - loss: 0.0031 - accuracy: 0.9992 - val_loss: 0.0441 - val_accuracy: 0.9916

In [31]:
y_pred = model.predict_classes(X_test)
y_pred

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=int32)

In [32]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [33]:
accuracy = accuracy_score(y_pred, y_test)
accuracy

0.9903069466882067

In [34]:
report = classification_report(y_pred, y_test)
print(report)

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      2440
           1       0.96      0.99      0.98       655

    accuracy                           0.99      3095
   macro avg       0.98      0.99      0.99      3095
weighted avg       0.99      0.99      0.99      3095



In [35]:
matrix = confusion_matrix(y_pred, y_test)
matrix

array([[2416,   24],
       [   6,  649]])